In [1]:
import numpy as np
import pandas as pd
import gurobipy as gp

## Problem 1

In [2]:
xy = pd.read_csv('problem1.csv')
nx = xy.shape[0]
nx

21

In [3]:
xy

,x,y
0,-2.0,4.353360
1,-1.8,3.200865
2,-1.6,2.342049
3,-1.4,2.138262
4,-1.2,1.516324
5,-1.0,1.295966
6,-0.8,0.591288
7,-0.6,0.616674
8,-0.4,0.244323
9,-0.2,0.069932


In [4]:
A = np.zeros((nx,nx))
b = np.zeros(nx)

## Loop for polynomial equation coefficient matrix

In [5]:
for row in range(nx):
    b[row] = xy.y[row]
    for power in range(nx):
        A[row,power] = xy.x[row]**power

In [6]:
A

array([[ 1.00000000e+00, -2.00000000e+00,  4.00000000e+00,
        -8.00000000e+00,  1.60000000e+01, -3.20000000e+01,
         6.40000000e+01, -1.28000000e+02,  2.56000000e+02,
        -5.12000000e+02,  1.02400000e+03, -2.04800000e+03,
         4.09600000e+03, -8.19200000e+03,  1.63840000e+04,
        -3.27680000e+04,  6.55360000e+04, -1.31072000e+05,
         2.62144000e+05, -5.24288000e+05,  1.04857600e+06],
       [ 1.00000000e+00, -1.80000000e+00,  3.24000000e+00,
        -5.83200000e+00,  1.04976000e+01, -1.88956800e+01,
         3.40122240e+01, -6.12220032e+01,  1.10199606e+02,
        -1.98359290e+02,  3.57046723e+02, -6.42684101e+02,
         1.15683138e+03, -2.08229649e+03,  3.74813368e+03,
        -6.74664062e+03,  1.21439531e+04, -2.18591156e+04,
         3.93464081e+04, -7.08235345e+04,  1.27482362e+05],
       [ 1.00000000e+00, -1.60000000e+00,  2.56000000e+00,
        -4.09600000e+00,  6.55360000e+00, -1.04857600e+01,
         1.67772160e+01, -2.68435456e+01,  4.29496730e

In [7]:
coefs = np.linalg.solve(A,b)

In [8]:
coefs

array([-2.68724962e-01,  4.25912462e-01,  1.46120424e+01, -7.64904952e+00,
       -1.60571678e+02,  4.44014690e+01,  8.28970249e+02, -1.23098065e+02,
       -2.17345669e+03,  1.84818348e+02,  3.17391444e+03, -1.60252343e+02,
       -2.71837186e+03,  8.22122451e+01,  1.38870524e+03, -2.45170060e+01,
       -4.14052554e+02,  3.91180137e+00,  6.61736781e+01, -2.57246750e-01,
       -4.36210853e+00])

In [9]:
xy.y[0]

4.35336047509895

In [10]:
xy.y[1]

3.2008647458938

In [11]:
y_at_neg2 = 0
for power in range(nx):
    y_at_neg2 += coefs[power]*(-2)**power
y_at_neg2

4.353360475040972

In [12]:
y_at_neg1p9 = 0
for power in range(nx):
    y_at_neg1p9 += coefs[power]*(-1.9)**power
y_at_neg1p9

772.2193696165923

## Fill with identity matrix below a certain point

In [13]:
test = np.diag(np.ones(10))
test[0:,:] = np.identity(10)
test

array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]])

I encourage you to look at a graph where you put dots at every x,y value from the data set in one color, and then create a finer dataset that has x values separated by 0.01 then evaluate y at each of those new x values using the polynomial you found above.  then plot a line that has those new x,y values in a different color.  You will see that in the middle of the graph the interpolation looks reasonable, but on the edges it's wildly oscilatory.

THis tells us that polynomial interpolation like this is terrible!

## Problem 2

In [14]:
obj = np.array([100,200,150,250,225])
lb = np.ones(5)*4
A = np.zeros((23+5,5))
sense = np.array(['']*(23+5))
b = np.zeros(23+5)

# Loop for x1 < = 2*x2

In [15]:
# not twice the hours...need all combinations of job comparisons! for each job compare to all other jobs
ind = list(range(5))
loc = 0

for i in ind:
    for j in ind: # no constraint that you can't be double yourself...loop over every value of ind except i
        if j!=i:
            A[loc,[i,j]] = [1,-2]
            sense[loc] = '<'
            loc += 1


In [16]:
# 40 hours per week
A[20,:] = 1
b[20] = 40
sense[20] = '='

In [17]:
# jobs 1 and 3 constraint
A[21,[0,2]] = 1
b[21] = 15
sense[21] = '>'

In [18]:
# jobs 4 and 5 constraint
A[22,[3,4]] = 1
b[22] = 15
sense[22] = '<'

In [19]:
A[23:,:] = np.identity(5) #fill coefficient matrix with identity matrix after a certain point
b[23:] = 4
sense[23:] = '>'



In [20]:
jobModel = gp.Model() # initialize an empty model

jobModX = jobModel.addMVar(5)#,lb=lb) # tell the model how many variables there are . #lb = Lower Bound
# must define the variables before adding constraints because variables go into the constraints
jobModCon = jobModel.addMConstrs(A, jobModX, sense, b) # add the constraints to the model
jobModel.setMObjective(None,obj,0,sense=gp.GRB.MAXIMIZE) # add the objective to the model...we'll talk about the None and the 0

jobModel.Params.OutputFlag = 0 # tell gurobi to shut up!!
jobModel.optimize()

Set parameter Username
Academic license - for non-commercial use only - expires 2023-09-14


In [21]:
jobModX.x

array([ 5., 10., 10., 10.,  5.])

In [22]:
jobModel.objval

7625.0

In [23]:
150*40
# the company makes $7625 per week from your work, so you are profitable

6000

## Calculate Shadow Price. 

How much will the income to company change by for a unit change in number of hours you've worked

In [24]:
jobModCon[20].Pi # 20 is the 40 hours per week constraint
# if they increase your hours to 41 they'll make $200 more dollars
# but you would have to be paid $225 for that hour
# they shouldn't ask you to work more!

175.0

In [25]:
A


array([[ 1., -2.,  0.,  0.,  0.],
       [ 1.,  0., -2.,  0.,  0.],
       [ 1.,  0.,  0., -2.,  0.],
       [ 1.,  0.,  0.,  0., -2.],
       [-2.,  1.,  0.,  0.,  0.],
       [ 0.,  1., -2.,  0.,  0.],
       [ 0.,  1.,  0., -2.,  0.],
       [ 0.,  1.,  0.,  0., -2.],
       [-2.,  0.,  1.,  0.,  0.],
       [ 0., -2.,  1.,  0.,  0.],
       [ 0.,  0.,  1., -2.,  0.],
       [ 0.,  0.,  1.,  0., -2.],
       [-2.,  0.,  0.,  1.,  0.],
       [ 0., -2.,  0.,  1.,  0.],
       [ 0.,  0., -2.,  1.,  0.],
       [ 0.,  0.,  0.,  1., -2.],
       [-2.,  0.,  0.,  0.,  1.],
       [ 0., -2.,  0.,  0.,  1.],
       [ 0.,  0., -2.,  0.,  1.],
       [ 0.,  0.,  0., -2.,  1.],
       [ 1.,  1.,  1.,  1.,  1.],
       [ 1.,  0.,  1.,  0.,  0.],
       [ 0.,  0.,  0.,  1.,  1.],
       [ 1.,  0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.],
       [ 0.,  0.,  1.,  0.,  0.],
       [ 0.,  0.,  0.,  1.,  0.],
       [ 0.,  0.,  0.,  0.,  1.]])

## Modified problem 2

In [26]:
obj = np.array([100,200,150,250,225,-100])
lb = np.array([4]*5+[0])
A = np.zeros((23,6))
sense = np.array(['']*23)
b = np.zeros(23)

In [27]:
# not twice the hours...need all combinations of job comparisons! for each job compare to all other jobs
ind = list(range(5))
loc = 0

for i in ind:
    for j in ind: # no constraint that you can't be double yourself...loop over every value of ind except i
        if j!=i:
            A[loc,[i,j]] = [1,-2]
            sense[loc] = '<'
            loc += 1


In [28]:
# 40 hours per week
A[20,0:5] = 1 # be careful not to include the slack variable in this constraint
b[20] = 40
sense[20] = '='

In [29]:
# jobs 1 and 3 constraint
A[21,[0,2,5]] = [1,1,-1]
b[21] = 12
sense[21] = '='

In [30]:
# jobs 4 and 5 constraint
A[22,[3,4,5]] = 1
b[22] = 15
sense[22] = '<'

In [31]:
jobModel2 = gp.Model() # initialize an empty model

jobMod2X = jobModel2.addMVar(6,lb=lb) # tell the model how many variables there are
# must define the variables before adding constraints because variables go into the constraints
jobMod2Con = jobModel2.addMConstrs(A, jobMod2X, sense, b) # add the constraints to the model
jobModel2.setMObjective(None,obj,0,sense=gp.GRB.MAXIMIZE) # add the objective to the model...we'll talk about the None and the 0

jobModel2.Params.OutputFlag = 0 # tell gurobi to shut up!!
jobModel2.optimize()

In [32]:
jobMod2X.x

array([ 6.5, 13. ,  6.5,  7.5,  6.5,  1. ])

In [33]:
np.sum(jobMod2X.x[:-1])

40.0

##  For Slack Calculation

In [34]:
A[22]

array([0., 0., 0., 1., 1., 1.])

In [40]:
np.dot(A[22,:],jobMod2X.x) - b[22]

0.0

15.0